In [1]:
import json
import cv2
import math

In [2]:
# frames for josn creation for accurate convention

frm_cnt = 270

frame_lists = []

for i in range(0, 1):
    objects = []
    frame_lists.append({"objects": objects})
    
edge_lists = []

for i in range(0, frm_cnt):
    edges = []
    edge_lists.append({"edges": edges})

In [3]:
def click_event(event, x, y, flags, params):
  
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
  
        # displaying the coordinates
        # on the Shell
        print(x, ' ', y)
  
        # displaying the coordinates
        # on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image, str(x) + ',' +
                    str(y), (x,y), font,
                    1, (255, 0, 0), 2)
        cv2.imshow('image', image)
  
    # checking for right mouse clicks     
    if event==cv2.EVENT_RBUTTONDOWN:
  
        # displaying the coordinates
        # on the Shell
        print(x, ' ', y)
  
        # displaying the coordinates
        # on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        b = image[y, x, 0]
        g = image[y, x, 1]
        r = image[y, x, 2]
        cv2.putText(image, str(b) + ',' +
                    str(g) + ',' + str(r),
                    (x,y), font, 1,
                    (255, 255, 0), 2)
        cv2.imshow('image', image)

In [4]:
image = cv2.imread('out.png',cv2.IMREAD_UNCHANGED)

In [5]:
busy_airports = {}
busy_airports = set(busy_airports)

with open('all_routes.txt') as f:
    lines = f.readlines()
    
    for each_line in lines:
        
        each_line = each_line.replace("\n", "")
        temp = each_line.split(" ")
        start, end, time_diff = temp[0], temp[1], temp[2] 
        busy_airports.add(start)
        busy_airports.add(end)

In [6]:
busy_airports

{'ARN',
 'ATL',
 'BOM',
 'CAI',
 'CDG',
 'CGK',
 'CPT',
 'CUN',
 'DEL',
 'DEN',
 'DFW',
 'DXB',
 'EZE',
 'FCO',
 'FRA',
 'GRU',
 'HKG',
 'HNL',
 'IST',
 'JFK',
 'LAS',
 'LAX',
 'LHR',
 'MAD',
 'MCO',
 'MEX',
 'MIA',
 'NRT',
 'ORD',
 'PEK',
 'PVG',
 'SEA',
 'SFO',
 'SIN',
 'SVO',
 'SYD',
 'YYC',
 'YYZ'}

In [7]:
MAP_WIDTH = 690
MAP_HEIGHT = 456 #1113

offset_x = 0
offset_y = 95

airport_dict = {}

airport_iter = 0

with open('airplane_cords.txt') as f:
    lines = f.readlines()
    
    for each_line in lines:
        #airport_iter += 1
        
        each_line = each_line.replace("\n", "")
        temp = each_line.split(" ")
        id, temp = temp[0], temp[1]
        
        #if id != "REK": continue
        if id not in busy_airports:
            #print("here",id)
            continue
            
        #print("passed",id)
        airport_iter += 1
        airport_dict[id] = airport_iter
        
        temp = temp.split(",")        
        # The order convension of coordinates follow the longitude-latitude-elevation order.
        # https://stackoverflow.com/questions/1369512/converting-longitude-latitude-to-x-y-on-a-map-with-calibration-points
        lon, lat = float(temp[0]), float(temp[1])
        x =  int((MAP_WIDTH/360.0) * (180 + lon))+offset_x
        y =  int((MAP_HEIGHT/180.0) * (90 - lat))+offset_y
                
        all_pt = []
        temp_pt = {"x": int(x), "y": int(y)}
        all_pt.append(temp_pt)
        frame_lists[0]["objects"].append({"id": airport_iter, "name": id, "bounds": all_pt})
        
        cv2.putText(image, id, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (209, 80, 0, 255), 3)         
        print(x, y, temp[0], temp[1],"id", airport_iter, "name", id)

203 220 -73.79168731973864 40.647672921902085 id 1 name JFK
178 269 -86.87973640933609 21.04449063988976 id 2 name CUN
191 257 -80.26496565324224 25.793010380998638 id 3 name MIA
126 193 -114.01090844101515 51.12673612021307 id 4 name YYC
614 232 140.39246737624134 35.76241570133719 id 5 name NRT
255 382 -46.48364975982179 -23.42005154321623 id 6 name GRU
578 244 121.78621380562439 31.150633439684313 id 7 name PVG
124 231 -115.14284782173708 36.08561338005141 id 8 name LAS
189 250 -81.2907857170215 28.42831453051457 id 9 name MCO
563 266 113.93338435186595 22.312419994845527 id 10 name HKG
544 319 103.99354166490136 1.3542935982235846 id 11 name SIN
344 192 -0.4632393123453369 51.47479515882837 id 12 name LHR
183 237 -84.4389921196544 33.63676942979369 id 13 name ATL
568 221 116.58636727543299 40.07586164417694 id 14 name PEK
484 274 72.86520869504798 19.092873887653894 id 15 name BOM
405 246 31.407928876106837 30.106456323838387 id 16 name CAI
380 409 18.60286463168987 -33.97369882123

In [8]:
with open('all_routes.txt') as f:
    lines = f.readlines()
    
    for each_line in lines:
        
        each_line = each_line.replace("\n", "")
        temp = each_line.split(" ")
        start, end, time_diff = temp[0], temp[1], temp[2] 
        
        frame_no = math.floor(float(time_diff)*10);        
        edge_lists[frame_no]["edges"].append({"edge_start":airport_dict[start], "edge_end":airport_dict[end], "weight":1})
        

In [9]:
cv2.imshow('image',image)

cv2.setMouseCallback('image', click_event)
 
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image

In [10]:
out_file = open("airplanes.json", "w")   
json.dump({"all_frame": frame_lists, "width": MAP_WIDTH, "height": MAP_HEIGHT, "node_type": "static"}, out_file)  
out_file.close() 

out_file = open("airplanes_routes.json", "w")   
json.dump({"edge_list": edge_lists}, out_file)  
out_file.close() 

In [11]:
print(airport_dict)

{'JFK': 1, 'CUN': 2, 'MIA': 3, 'YYC': 4, 'NRT': 5, 'GRU': 6, 'PVG': 7, 'LAS': 8, 'MCO': 9, 'HKG': 10, 'SIN': 11, 'LHR': 12, 'ATL': 13, 'PEK': 14, 'BOM': 15, 'CAI': 16, 'CPT': 17, 'DEL': 18, 'DEN': 19, 'DFW': 20, 'DXB': 21, 'EZE': 22, 'IST': 23, 'LAX': 24, 'MEX': 25, 'HNL': 26, 'ORD': 27, 'SEA': 28, 'SFO': 29, 'SVO': 30, 'YYZ': 31, 'SYD': 32, 'CDG': 33, 'FRA': 34, 'FCO': 35, 'MAD': 36, 'ARN': 37, 'CGK': 38}
